# nearest neighbor preserverance
1. knn preserving evaluation

## load data

In [1]:
import torch
import math
import numpy as np

In [2]:
# hyperparameters
data_shape = (2048,)

n_hidden = 500
dim_img = 2048  # number of pixels for a gap layer
dim_z = 2

# train
n_epochs = 500
batch_size = 200
learn_rate = 0.001

### train_data

In [3]:
train_data = np.load("train_data.npy")
train_pred_labels = np.load("train_pred_labels.npy")
train_data.shape, train_pred_labels.shape

((50000, 2048), (50000,))

### test_data

In [4]:
test_data = np.load("test_data.npy")
test_pred_labels = np.load("test_pred_labels.npy")
test_data.shape, test_pred_labels.shape

((10000, 2048), (10000,))

## border preserving before DBP
1. M_<CP,CP>(original)
2. M_<CP,CP>(core)
3. M_<CP,CP>(core) rho=0
4. M_<CP,DP>(core)
5. M_<CP,CP><CP,DP>(core)
6. M_<bw_Complex>(core) k=15
7. M_<bw_Complex>(core) k=30

In [5]:
# define encoder
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# test encoder/decoder		

### M_<CP,CP>(original)

In [6]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_tptp")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_tptp


#### train

In [7]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [8]:
low_train = load_encoder(train_data).cpu().numpy()

Instructions for updating:
Use tf.identity instead.


In [9]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

10 0.9842283997639268
20 0.9842833103993436
30 0.984340082635198
50 0.9844427716732266


In [9]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:28:30 2021 Building RP forest with 16 trees
Mon Jan 25 13:28:33 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 13:28:44 2021 Building RP forest with 16 trees
Mon Jan 25 13:28:45 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.93836, 10.0, 1.0)

In [10]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:28:46 2021 Building RP forest with 16 trees
Mon Jan 25 13:28:49 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:28:51 2021 Building RP forest with 16 trees
Mon Jan 25 13:28:51 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.921, 19.0, 1.0)

In [11]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:28:53 2021 Building RP forest with 16 trees
Mon Jan 25 13:28:56 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:29:00 2021 Building RP forest with 16 trees
Mon Jan 25 13:29:00 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.46464, 28.0, 1.0)

In [12]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:29:03 2021 Building RP forest with 16 trees
Mon Jan 25 13:29:05 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:29:17 2021 Building RP forest with 16 trees
Mon Jan 25 13:29:17 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.65164, 46.0, 1.0)

#### test

In [13]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [14]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [15]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [16]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:29:26 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:29 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 13:29:31 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:31 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.9149, 9.0, 1.0)

In [17]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:29:32 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:36 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:29:38 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:39 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.8138, 19.0, 1.0)

In [18]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:29:40 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:44 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:29:48 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:49 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.2684, 28.0, 1.0)

In [19]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:29:51 2021 Building RP forest with 17 trees
Mon Jan 25 13:29:54 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:30:08 2021 Building RP forest with 17 trees
Mon Jan 25 13:30:08 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.2468, 47.0, 1.0)

### M_<CP,CP>(core)

In [10]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tptp")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tptp


#### train

In [11]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [12]:
low_train = load_encoder(train_data).cpu().numpy()

In [13]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

10 0.9848173464574018
20 0.9848220433064169
30 0.984871325933266
50 0.9849687419683723


In [9]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:32:07 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:10 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 13:32:22 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:22 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.92736, 10.0, 1.0)

In [10]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:32:23 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:26 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:32:28 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:29 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.90048, 19.0, 1.0)

In [11]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:32:31 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:33 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:32:37 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:38 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.41748, 28.0, 1.0)

In [12]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:32:40 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:43 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:32:54 2021 Building RP forest with 16 trees
Mon Jan 25 13:32:55 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.58976, 46.0, 1.0)

#### test

In [13]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [14]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [15]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [16]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:33:03 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:07 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 13:33:08 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:08 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.9149, 9.0, 1.0)

In [17]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:33:10 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:13 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:33:16 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:16 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.797, 18.0, 1.0)

In [18]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:33:18 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:22 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:33:26 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:26 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.2076, 28.0, 1.0)

In [19]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:33:29 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:32 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:33:46 2021 Building RP forest with 17 trees
Mon Jan 25 13:33:46 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.1519, 46.0, 1.0)

### M_<CP,CP>(core) rho=0

In [14]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tptp_rho0")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tptp_rho0


#### train

In [15]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [16]:
low_train = load_encoder(train_data).cpu().numpy()

In [17]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

10 0.9849535878122218
20 0.9849754818439248
30 0.9849990296503152
50 0.9850609150617432


In [23]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:35:54 2021 Building RP forest with 16 trees
Mon Jan 25 13:35:57 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 13:35:58 2021 Building RP forest with 16 trees
Mon Jan 25 13:35:58 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.96362, 9.0, 1.0)

In [24]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:36:00 2021 Building RP forest with 16 trees
Mon Jan 25 13:36:03 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:36:05 2021 Building RP forest with 16 trees
Mon Jan 25 13:36:05 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.00096, 19.0, 1.0)

In [25]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:36:07 2021 Building RP forest with 16 trees
Mon Jan 25 13:36:10 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:36:14 2021 Building RP forest with 16 trees
Mon Jan 25 13:36:14 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.59022, 29.0, 1.0)

In [26]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:36:17 2021 Building RP forest with 16 trees
Mon Jan 25 13:36:19 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:36:31 2021 Building RP forest with 16 trees
Mon Jan 25 13:36:31 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.87984, 46.0, 1.0)

#### test

In [27]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [28]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [29]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [30]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:36:40 2021 Building RP forest with 17 trees
Mon Jan 25 13:36:44 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 13:36:45 2021 Building RP forest with 17 trees
Mon Jan 25 13:36:46 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.9425, 9.0, 1.0)

In [31]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:36:47 2021 Building RP forest with 17 trees
Mon Jan 25 13:36:50 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:36:53 2021 Building RP forest with 17 trees
Mon Jan 25 13:36:53 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.8773, 19.0, 1.0)

In [32]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:36:55 2021 Building RP forest with 17 trees
Mon Jan 25 13:36:58 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:37:03 2021 Building RP forest with 17 trees
Mon Jan 25 13:37:03 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.333, 27.0, 1.0)

In [33]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 13:37:06 2021 Building RP forest with 17 trees
Mon Jan 25 13:37:09 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 13:37:23 2021 Building RP forest with 17 trees
Mon Jan 25 13:37:23 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.3911, 48.0, 1.0)

### M_<CP,DP>(core)

In [6]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tpdp")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tpdp


#### train

In [7]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [8]:
low_train = load_encoder(train_data).cpu().numpy()

Instructions for updating:
Use tf.identity instead.


In [21]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

10 0.9765683108963779
20 0.9765955240896947
30 0.9766400976888969
50 1.0111850059189376


In [9]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:18:06 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:09 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:18:23 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:23 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.68632, 9.0, 1.0)

In [10]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:18:25 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:28 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:18:30 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:30 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.08414, 19.0, 1.0)

In [11]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:18:33 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:35 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:18:40 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:40 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.87756, 28.0, 1.0)

In [12]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:18:43 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:46 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:18:58 2021 Building RP forest with 16 trees
Mon Jan 25 17:18:58 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(9.27474, 45.0, 1.0)

#### test

In [13]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [14]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [15]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [25]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,t)

10 0.9758711358211982


TypeError: 'numpy.float64' object is not callable

In [16]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:21:27 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:31 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:21:32 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:32 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.754, 9.0, 1.0)

In [17]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:21:34 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:37 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:21:40 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:40 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.1878, 18.0, 1.0)

In [18]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:21:42 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:45 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:21:50 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:50 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.9968, 27.0, 1.0)

In [19]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:21:52 2021 Building RP forest with 17 trees
Mon Jan 25 17:21:56 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:22:10 2021 Building RP forest with 17 trees
Mon Jan 25 17:22:10 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(9.3461, 45.0, 1.0)

### M_<CP,CP><CP,DP>(core)

In [20]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_tptptpdp")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_tptptpdp


#### train

In [21]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [22]:
low_train = load_encoder(train_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [23]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:22:33 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:36 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:22:37 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:37 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.92228, 10.0, 1.0)

In [24]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:22:39 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:42 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:22:44 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:44 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.8684, 19.0, 1.0)

In [25]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:22:46 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:49 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:22:53 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:53 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.3445, 28.0, 1.0)

In [26]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:22:56 2021 Building RP forest with 16 trees
Mon Jan 25 17:22:59 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:23:11 2021 Building RP forest with 16 trees
Mon Jan 25 17:23:11 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(12.40648, 46.0, 1.0)

#### test

In [27]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [28]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [29]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [30]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:23:19 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:23 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:23:25 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:25 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.9095, 9.0, 1.0)

In [31]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:23:26 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:30 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:23:32 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:33 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.7786, 18.0, 1.0)

In [32]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:23:34 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:38 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:23:43 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:43 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(6.1705, 28.0, 1.0)

In [33]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:23:45 2021 Building RP forest with 17 trees
Mon Jan 25 17:23:49 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:24:03 2021 Building RP forest with 17 trees
Mon Jan 25 17:24:03 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(11.9782, 47.0, 1.0)

### M_<new_Complex, k=15> (core)

In [34]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_complex_15")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_complex_15


#### train

In [35]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [36]:
low_train = load_encoder(train_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [37]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:24:30 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:32 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:24:34 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:34 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.58952, 9.0, 1.0)

In [38]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:24:36 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:38 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:24:40 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:41 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(2.80488, 17.0, 1.0)

In [39]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:24:43 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:45 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:24:49 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:50 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.36568, 27.0, 1.0)

In [40]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:24:52 2021 Building RP forest with 16 trees
Mon Jan 25 17:24:55 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:25:07 2021 Building RP forest with 16 trees
Mon Jan 25 17:25:07 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(8.20522, 45.0, 1.0)

#### test

In [41]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [42]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [43]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [44]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:25:16 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:20 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:25:21 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:22 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.6802, 9.0, 1.0)

In [45]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:25:23 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:27 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:25:29 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:30 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(2.9823, 19.0, 1.0)

In [46]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:25:31 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:35 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:25:40 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:40 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.604, 24.0, 1.0)

In [47]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:25:42 2021 Building RP forest with 17 trees
Mon Jan 25 17:25:46 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:26:00 2021 Building RP forest with 17 trees
Mon Jan 25 17:26:00 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(8.4895, 44.0, 1.0)

### M_<new_Complex, k=30> (core)

In [48]:
import os
save_location = "parametric_umap_models\dbp_exp"
# load model
# load encoder
encoder_output = os.path.join(save_location, "encoder_core_complex_30")
if os.path.exists(encoder_output):
    load_encoder = tf.keras.models.load_model(encoder_output)
    print("Keras encoder model loaded from {}".format(encoder_output))

Keras encoder model loaded from parametric_umap_models\dbp_exp\encoder_core_complex_30


#### train

In [49]:
# number of trees in random projection forest
n_trees = 5 + int(round((train_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(train_data.shape[0]))))
# distance metric
metric = "euclidean"

In [50]:
low_train = load_encoder(train_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    t = trustworthiness(train_data, low_train, n_neighbors=int(k), metric='euclidean')
    print(k,t)

In [51]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=10
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:26:22 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:25 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:26:26 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:26 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.48556, 10.0, 1.0)

In [52]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_train,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:26:28 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:31 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:26:33 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:33 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(2.50754, 19.0, 1.0)

In [53]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:26:35 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:38 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:26:41 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:42 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(3.82668, 29.0, 1.0)

In [54]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    train_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
low_train = load_encoder(train_data).cpu().numpy()
nnd = NNDescent(
    low_train,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph
border_pres = np.zeros(len(train_data))
for i in range(len(train_data)):
    border_pres[i] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:26:44 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:47 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:26:59 2021 Building RP forest with 16 trees
Mon Jan 25 17:26:59 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(7.07654, 49.0, 1.0)

#### test

In [55]:
fitting_data =  np.concatenate((train_data, test_data), axis=0)

In [56]:
# number of trees in random projection forest
n_trees = 5 + int(round((fitting_data.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(fitting_data.shape[0]))))
# distance metric
metric = "euclidean"

In [57]:
low_fitting = load_encoder(fitting_data).cpu().numpy()

In [ ]:
from sklearn.manifold import trustworthiness
knns = [10,20,30,50]
for k in knns:
    trustworthiness = trustworthiness(fitting_data, low_fitting, n_neighbors=k, metric='euclidean')
    print(k,trustworthiness)

In [58]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=10,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:27:08 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:11 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	Stopping threshold met -- exiting after 3 iterations
Mon Jan 25 17:27:13 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:13 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(1.6062, 9.0, 1.0)

In [59]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=20,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=20
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:27:14 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:18 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:27:21 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:21 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(2.7418, 18.0, 1.0)

In [60]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=30,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=30
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:27:23 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:26 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:27:31 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:31 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(4.1855, 27.0, 1.0)

In [61]:
from pynndescent import NNDescent
# get nearest neighbors
nnd = NNDescent(
    fitting_data,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
high_ind, _ = nnd.neighbor_graph
nnd = NNDescent(
    low_fitting,
    n_neighbors=50,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
low_ind, _ = nnd.neighbor_graph

border_pres = np.zeros(len(test_data))
train_num = len(train_data)
test_num = len(fitting_data)
for i in range(train_num,test_num,1):
    border_pres[i-train_num] = len(np.intersect1d(high_ind[i],low_ind[i]))
# tptp K=50
border_pres.mean(), border_pres.max(),border_pres.min()

Mon Jan 25 17:27:33 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:37 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations
Mon Jan 25 17:27:50 2021 Building RP forest with 17 trees
Mon Jan 25 17:27:51 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	Stopping threshold met -- exiting after 2 iterations


(7.6436, 44.0, 1.0)